In [1]:
from msilib.schema import Class
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import time
startTime = time.time()

df = pd.read_csv('Cars93.csv')
df.head()

,Unnamed: 0,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,Driver & Passenger,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


In [2]:
#check nulls for each column
print(df.isna().sum())
df.info()

Unnamed: 0             0
Manufacturer           0
Model                  0
Type                   0
Min.Price              0
Price                  0
Max.Price              0
MPG.city               0
MPG.highway            0
AirBags                0
DriveTrain             0
Cylinders              0
EngineSize             0
Horsepower             0
RPM                    0
Rev.per.mile           0
Man.trans.avail        0
Fuel.tank.capacity     0
Passengers             0
Length                 0
Wheelbase              0
Width                  0
Turn.circle            0
Rear.seat.room         2
Luggage.room          11
Weight                 0
Origin                 0
Make                   0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          93 non-null     int64  
 1   Manufacturer        93 non-null 

In [3]:
df.drop(columns=['Unnamed: 0','Manufacturer','Model','Type','Min.Price','Max.Price','AirBags','DriveTrain','Rev.per.mile','Passengers','Length','Wheelbase','Width','Turn.circle','Rear.seat.room','Luggage.room','Origin','Make'], axis=1, inplace=True)
df.head()

,Price,MPG.city,MPG.highway,Cylinders,EngineSize,Horsepower,RPM,Man.trans.avail,Fuel.tank.capacity,Weight
0,15.9,25,31,4,1.8,140,6300,Yes,13.2,2705
1,33.9,18,25,6,3.2,200,5500,Yes,18.0,3560
2,29.1,20,26,6,2.8,172,5500,Yes,16.9,3375
3,37.7,19,26,6,2.8,172,5500,Yes,21.1,3405
4,30.0,22,30,4,3.5,208,5700,Yes,21.1,3640


In [4]:
print(df.isna().sum())

Price                 0
MPG.city              0
MPG.highway           0
Cylinders             0
EngineSize            0
Horsepower            0
RPM                   0
Man.trans.avail       0
Fuel.tank.capacity    0
Weight                0
dtype: int64


In [5]:
df.drop(df[df['Cylinders'] == 'rotary'].index, inplace = True)

In [6]:
numericData = df.drop(columns='Man.trans.avail').copy()
# Man.trans.avail as category column
categoricalData = df['Man.trans.avail'].copy()

categoricalData = categoricalData.map({'Yes': 1, 'No': 0})
# Make it the column is predicts

In [7]:
df = pd.concat([numericData, categoricalData], axis=1)

In [8]:
x = numericData
x_scaled = preprocessing.scale(x)
x = x_scaled
y = categoricalData
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=0)

In [9]:
Classifier = SVC(kernel='rbf', C=2, gamma='scale')
Classifier.fit(x_train, y_train)
y_pred = Classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)*100
confusion_mat = confusion_matrix(y_test, y_pred)
score = Classifier.score(x_train, y_train)
cv_scores = cross_val_score(Classifier, x_train, y_train, cv=10)
cr = classification_report(y_test, y_pred)

In [10]:
print(df.corr())
print('---------------------------------------------------------')
print("Accuracy for SVM is: ", accuracy)
print('---------------------------------------------------------')
print("Confusion Matrix")
print(confusion_mat)
print('---------------------------------------------------------')
print("Score: ", score)
print('---------------------------------------------------------')
print("CV average score: %.2f" % cv_scores.mean())
print('---------------------------------------------------------')
print('Classification Report:')
print(cr)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

                       Price  MPG.city  MPG.highway  EngineSize  Horsepower  \
Price               1.000000 -0.589267    -0.556696    0.629168    0.784143   
MPG.city           -0.589267  1.000000     0.943668   -0.734536   -0.670457   
MPG.highway        -0.556696  0.943668     1.000000   -0.646166   -0.618626   
EngineSize          0.629168 -0.734536    -0.646166    1.000000    0.790067   
Horsepower          0.784143 -0.670457    -0.618626    0.790067    1.000000   
RPM                -0.036347  0.395603     0.339630   -0.535751   -0.011519   
Fuel.tank.capacity  0.614047 -0.811147    -0.784466    0.786021    0.709805   
Weight              0.658537 -0.850998    -0.816242    0.849277    0.765391   
Man.trans.avail    -0.341525  0.455347     0.412995   -0.632104   -0.380429   

                         RPM  Fuel.tank.capacity    Weight  Man.trans.avail  
Price              -0.036347            0.614047  0.658537        -0.341525  
MPG.city            0.395603           -0.811147 -0.8